In [10]:
import torch
#import torchvision
#import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import NslKdd as kdd

In [11]:
transform=torch.FloatTensor

#dataset
trainset=kdd.NslKddset("../../data","KDDTrain+",train=True,transform=transform)
testset=kdd.NslKddset("../../data","KDDTest+",train=False,transform=transform)

#dataloaders
trainloader=torch.utils.data.DataLoader(trainset,batch_size=10,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=10)

#constant for classes
classes=('normaly','anamaly')

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv1d(1,10,2)
        self.maxpool=nn.MaxPool1d(2,stride=2)
        self.avgpool=nn.AvgPool1d(2,stride=2)
        self.conv2=nn.Conv1d(10,20,2)
        self.fc1=nn.Linear(9*20,80)
        self.fc2=nn.Linear(80,16)
        self.fc3=nn.Linear(16,2)

    def forward(self,x):
        x=F.relu(self.avgpool(self.conv1(x)))
        x=F.relu(self.maxpool(self.conv2(x)))
        x=x.view(-1,9*20)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x


In [13]:
net = Net()
criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, 3,gamma=0.1)

In [14]:
running_loss = 0.0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            scheduler.step()
            print("[%d,%6d] ava_loss: %.5f"%(epoch+1,i+1,running_loss/1000))
            running_loss = 0.0
print('Finished Training')

[1,  1000] ava_loss: 4.92340
[1,  2000] ava_loss: 0.09650
[1,  3000] ava_loss: 0.08542
[1,  4000] ava_loss: 0.06792
[1,  5000] ava_loss: 0.06454
[1,  6000] ava_loss: 0.05903
[1,  7000] ava_loss: 0.05768
[1,  8000] ava_loss: 0.05822
[1,  9000] ava_loss: 0.05749
[1, 10000] ava_loss: 0.05795
[1, 11000] ava_loss: 0.05684
[1, 12000] ava_loss: 0.05675
Finished Training


In [15]:
PATH="../../pth/nsl_kdd_test.pth"
torch.save(net.state_dict(),PATH)

In [16]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        cvdata, labels = data
        outputs = net(cvdata)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("correct:%d  total:%d" %(correct,total))
print('Accuracy of the network: %d %%' % (
    100 * correct / total))

correct:16895  total:22544
Accuracy of the network: 74 %


In [18]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        cvdata, labels = data
        outputs = net(cvdata)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
       
        for i in range(len(labels)):
            label = labels[i]
            class_correct[int(label)] += c[i].item()
            class_total[int(label)] += 1

for i in range(2):
    print("correct:%d  total:%d" %(class_correct[i],class_total[i]))
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

correct:9431  total:9711
Accuracy of normaly : 97.116672 %
correct:7464  total:12833
Accuracy of anamaly : 58.162550 %
